In [1]:

import sys
from pathlib import Path

# Add parent directory (src/) to sys.path
sys.path.append(str(Path.cwd().parent))

from Data_Handler import get_data
import json
import os
import zipfile
from collections import Counter
from typing import Dict
import shutil
import sqlite3, json, os
import pandas as pd
from pathlib import Path
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
from timeit import default_timer as timer
from tqdm.auto import tqdm
import torch.optim as optim
import matplotlib.pyplot as plt
from torchmetrics.classification import MulticlassAccuracy
import optuna
import torch.optim as optim

/home/fillies/Documents/moon/kilter/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
!pip install optuna

In [3]:
# loading in a specific board
data = get_data(board_name="12 x 12 with kickboard Square")
placements = data["placements"]
roles = data["roles"]

placements.describe(), roles.describe()



(            route_id     difficulty        hold_id      token_num  \
 count  452057.000000  452057.000000  452057.000000  452057.000000   
 mean   106045.795484    1242.255280     265.403715    1304.119410   
 std     65642.474089       4.485806     130.137169     130.992049   
 min         2.000000    1233.000000       3.000000    1073.000000   
 25%     47390.000000    1239.000000     176.000000    1199.000000   
 50%    105517.000000    1243.000000     277.000000    1287.000000   
 75%    162363.000000    1246.000000     363.000000    1389.000000   
 max    222622.000000    1253.000000     526.000000    1599.000000   
 
                    x              y         set_id  
 count  452057.000000  452057.000000  452057.000000  
 mean       72.420106      78.156463       5.165099  
 std        31.705649      43.410323       7.860596  
 min         4.000000       4.000000       1.000000  
 25%        48.000000      44.000000       1.000000  
 50%        72.000000      80.000000       1

In [4]:
# get the id of each hold in the 
holds_per_route = (
    placements.groupby("route_id")
    .agg(
        difficulty=("difficulty", "first"),
        hold_ids=("hold_id", list)
        )
    .reset_index()
)

# filer for routs with less than n holds
num_holds = 20
holds_per_route = holds_per_route[holds_per_route["hold_ids"].apply(len)< num_holds]

# subtracht difficulty offset
holds_per_route["difficulty"] = holds_per_route["difficulty"] - holds_per_route["difficulty"].min()
holds_per_route


,route_id,difficulty,hold_ids
0,2,6,"[42, 156, 94, 169, 115, 201, 84, 181, 68, 184,..."
1,5,7,"[432, 517, 349, 409, 297, 127, 241, 450, 228, ..."
2,10,8,"[395, 321, 430, 349, 410, 355, 19, 443, 336, 3..."
3,13,11,"[278, 324, 493, 262, 344, 373, 461, 481, 412, ..."
4,16,10,"[104, 223, 151, 261, 346, 372, 461, 353, 191, ..."
...,...,...,...
36474,222597,1,"[154, 210, 150, 227, 173, 287, 232, 255, 238, ..."
36475,222600,15,"[332, 436, 340, 378, 172, 259, 373, 175, 178, ..."
36476,222614,12,"[270, 382, 397, 289, 372, 176, 182, 134, 291]"
36477,222618,17,"[493, 341, 172, 236, 77, 108]"


In [5]:
mlb = MultiLabelBinarizer()

one_hot = pd.DataFrame(
    mlb.fit_transform(holds_per_route["hold_ids"]),
    columns=mlb.classes_,
    index=holds_per_route["route_id"]
).reset_index()

# Add back difficulty column
one_hot = one_hot.merge(
    holds_per_route[["route_id", "difficulty"]],
    on="route_id"
)

one_hot



,route_id,3,4,5,6,7,8,9,10,11,...,518,519,520,521,522,523,524,525,526,difficulty
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,6
1,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,7
2,10,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,8
3,13,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,11
4,16,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35005,222597,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
35006,222600,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,15
35007,222614,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,12
35008,222618,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,17


In [6]:

# go up from src/ to project root, then into data/
project_root = Path.cwd().parent.parent   # if running from inside src/
data_dir = project_root / "data"

# make sure it exists
data_dir.mkdir(exist_ok=True)

# save one-hot CSV in data/
out_path = data_dir / "routes_onehot.csv"
one_hot.to_csv(out_path, index=False)

print(f"Saved to {out_path}")

Saved to /home/fillies/Documents/moon/kilter/data/routes_onehot.csv


In [7]:
from sklearn.preprocessing import OneHotEncoder
# Suppose 'difficulty' is the target
X = one_hot.drop(columns=["difficulty", "route_id"])
y = one_hot[["difficulty"]]




# also encode y in one hot for loss

enc = OneHotEncoder(sparse_output=False)
y = enc.fit_transform(y)

# Make a DataFrame for readability
y = pd.DataFrame(
    y,
    columns=[f"difficulty_{cls}" for cls in enc.categories_[0]],
    index=one_hot.index
)

print(y.shape)





(35010, 21)


# Starting of Moddel:

In [8]:
from utils import EarlyStopping, plot_training_history
from models import ShallowMLP
from trainer import train_model



torch.__version__
device = "cuda" if torch.cuda.is_available() else "cpu"
device


'cpu'

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.2, # 20% test, 80% train
                                                    random_state=42)

X_train = torch.tensor(X_train.values, dtype=torch.float32).to(device)
X_test  = torch.tensor(X_test.values, dtype=torch.float32).to(device)
y_train = torch.tensor(y_train.values, dtype=torch.float32).to(device)
y_test  = torch.tensor(y_test.values, dtype=torch.float32).to(device)

train_data = TensorDataset(X_train, y_train)
test_data = TensorDataset(X_test, y_test)

batch_size = 64
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size)


train_features_batch, train_labels_batch = next(iter(train_loader))
train_features_batch.shape, train_labels_batch.shape

(torch.Size([64, 476]), torch.Size([64, 21]))

In [10]:
shallowMLP = ShallowMLP(input_dim=X_test.shape[1],
                        hidden_dim=32,
                        num_classes= y_test.shape[1],
                        drop_out=0.3
                        ).to(device)

loss_fn = torch.nn.BCEWithLogitsLoss()
acc_fn = MulticlassAccuracy(num_classes=21).to(device)
optimizer = optim.Adam(shallowMLP.parameters(), lr=1e-3, weight_decay=1e-5 )

"""model, history = train_model(
    model=shallowMLP,
    train_loader=train_loader,
    test_loader=test_loader,
    loss_fn=loss_fn,
    optimizer=optimizer,
    acc_fn=acc_fn,
    device=device,
    epochs=1000,
    patience=10
)"""




'model, history = train_model(\n    model=shallowMLP,\n    train_loader=train_loader,\n    test_loader=test_loader,\n    loss_fn=loss_fn,\n    optimizer=optimizer,\n    acc_fn=acc_fn,\n    device=device,\n    epochs=1000,\n    patience=10\n)'

In [11]:
#plot_training_history(history)

In [12]:
def objective(trial: optuna.Trial):
    # --- Suggest hyperparameters ---
    hidden_dim = trial.suggest_categorical("hidden_dim", [16, 32, 64, 128, 256])
    dropout = trial.suggest_categorical("dropout", [0.1, 0.2, 0.3, 0.5])
    lr = trial.suggest_float("lr", 1e-5, 1e-2, log=True)   # log scale
    weight_decay = trial.suggest_float("weight_decay", 1e-6, 1e-3, log=True)

    # --- Build model ---
    model = ShallowMLP(
        input_dim=X_train.shape[1],
        hidden_dim=hidden_dim,
        num_classes=y_train.shape[1],
        drop_out=dropout
    ).to(device)

    loss_fn = torch.nn.BCEWithLogitsLoss()
    acc_fn = MulticlassAccuracy(num_classes=y_train.shape[1]).to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    
    checkpoint_path = f"checkpoints/model_hd{hidden_dim}_do{dropout:.2f}_lr{lr:.4f}_wd{weight_decay:.6f}.pt"


    # --- Train for fewer epochs (fast search) ---
    model, history = train_model(
        model=model,
        train_loader=train_loader,
        test_loader=test_loader,
        loss_fn=loss_fn,
        optimizer=optimizer,
        acc_fn=acc_fn,
        device=device,
        epochs=50,      # use small value for tuning
        patience=5,
        checkpoint_path=checkpoint_path
    )

    # --- Return final test accuracy ---
    return history["test_acc"][-1]


In [ ]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=30)

print("Best trial:")
print("  Value:", study.best_trial.value)
print("  Params:", study.best_trial.params)


[I 2025-09-18 09:41:13,090] A new study created in memory with name: no-name-88f24ed6-b1ba-4ff5-bda8-dc7c2bd5524d
